# Implementación manual de analizadores léxicos y manejo de estructuras de información
Notas de clase sobre Teoría de la Compilación

**Juan David Velásquez Henao**   
jdvelasq@unal.edu.co  
Universidad Nacional de Colombia, Sede Medellín  
Facultad de Minas  
Medellín, Colombia  

[Licencia](https://github.com/jdvelasq/teoria-de-la-compilacion/blob/master/LICENCIA.txt)  
[Readme](https://github.com/jdvelasq/teoria-de-la-compilacion/blob/master/readme.md)

**Software utilizado**.

> Este es un documento interactivo escrito como un notebook de [Jupyter](http://jupyter.org), en el cual se presenta una introducción al diseño de lectores, generadores, traductores, interpretes y compiladores. Los notebooks de Jupyter permiten incoporar simultáneamente código, texto, gráficos y ecuaciones. El código presentado en este notebook puede ejecutarse en los sistemas operativos Windows, Linux y OS X.

> Haga click [aquí](https://github.com/jdvelasq/guias-de-instalacion) para obtener instrucciones detalladas sobre como instalar Jupyter en Windows y Mac OS X.

> Haga clic [aquí](http://nbviewer.jupyter.org/github/jdvelasq/teoria-de-la-compilacion/blob/master/tcc-04-analisis-lexico.ipynb) para ver la última versión de este documento en nbviewer.

> Descargue la última versión de este documento, los archivos de datos y los programas en Python a su disco duro; luego, carguelos y ejecutelos en línea en [Try Jupyter!](https://try.jupyter.org)

#### Contenido

>   * [dataTree.py](#dataTree.py)
    * [Propiedades y funciones](#Propiedades-y-funciones)
    * [Ejemplos](#Ejemplos)
  * [shlex - Herramienta para construir analizadores léxicos sencillos](#shlex---Herramienta-para-construir-analizadores-léxicos-sencillos)
  * [yylex.py - Analizador léxico para el lenguaje de prueba](#yylex.py---Analizador-léxico-para-el-lenguaje-de-prueba)
    * [Gramática básica](#Gramática-básica.)
  * [Ejemplos lenguaje bcc](#Ejemplos-lenguaje-bcc)
  * [Gramática extendida (lenguaje final)](#Gramática-extendida-para-el-lenguaje-final)

# dataTree.py

[Contenido](#Contenido)

Es una estructura de datos, tipo arbol, que permite almacenar información en un formato similar al XML. La estructura base es el nodo.

## Propiedades y funciones

[Contenido](#Contenido)

**`tag`** -- Es un string que representa el nombre clave del nodo.  

**`attrib`** -- Es un diccionario que permite almacenar las propiedades de cada nodo.  

**`children`** -- Es una lista que contiene los nodos hijos del nodo actual.  

**`text`** -- Es un string que permite almacenar una cadena de texto adicional al tag.   

In [1]:
%%sh
pygmentize -O linenos=1 -g dataTree.py

0001: ###< 2016-08-28 17:04:40.181913 >###
0002: 
0003: """ DataTree for Python
0004: 
0005: This module implements a generic tree data structure for storing and
0006: retrieving hierarchical information.
0007: 
0008: This data structure is used to store the information generated
0009: during the compilation process.
0010: """
0011: 
0012: class TreeNode:
0013:     def __init__(self, tag, attrib = {}):
0014:         self.tag = tag
0015:         self.attrib = attrib.copy()
0016:         self._children = []
0017:         self.text = None
0018: 
0019:     def __repr__(self):
0020:         return '<Node {}>'.format(repr(self.tag))
0021: 
0022:     # methods operating over subelements
0023:     def __len__(self):
0024:         return len(self._children)
0025: 
0026:     def __iter__(self):
0027:         return self._children.__iter__()
0028: 
0029:     def __getitem__(self, index):
0030:         return self._children[index]
0031: 
0032:     def __setitem__(self, index, value):
0033:        

## Ejemplos

[Contenido](#Contenido)

**Ejemplo**. Cree el siguiente árbol usando `dataTree.py`:

```
+-- ROOT
     +-- A
     |   +-- A1
     |   +-- A2
     |   +-- A3
     +-- B
     |   +-- B1
     |   +-- B2
     +-- C
         +-- C1
         +-- C2
         +-- C3 {a: 1, b: 2, c: 3}
         +-- C4 {d: 4, e: 5, f: 6}
```

In [2]:
import dataTree as dt

root = dt.TreeNode('ROOT')

# todos son hijos del nodo raiz
A = dt.SubNode(root, 'A')
B = dt.SubNode(root, 'B')
C = dt.SubNode(root, 'C')

# hijos del nodo A
A1 = dt.SubNode(A, 'A1')
A2 = dt.SubNode(A, 'A2')
A3 = dt.SubNode(A, 'A3')


# hijos del nodo B
B1 = dt.SubNode(B, 'B1')
B2 = dt.SubNode(B, 'B2')

# hijos del nodo C
C1 = dt.SubNode(C, 'C1')
C2 = dt.SubNode(C, 'C2')
C3 = dt.SubNode(C, 'C3', dict(a=1, b=2, c=3))
C4 = dt.SubNode(C, 'C4', {'d':4, 'e':5, 'f':6})

dt.printTree(root)

+-- ROOT
    +-- A
    |   +-- A1
    |   +-- A2
    |   +-- A3
    +-- B
    |   +-- B1
    |   +-- B2
    +-- C
        +-- C1
        +-- C2
        +-- C3 {a: 1, b: 2, c: 3}
        +-- C4 {d: 4, e: 5, f: 6}


**Ejemplo**. Salve el arbol al disco y recuperlo en otra variable.

In [3]:
# lo salva al disco duro
import pickle
f = open('example.dataTree', 'wb')
pickle.dump(root, f)
f.close()

# lo recupera del disco duro
f = open('example.dataTree', 'rb')
data = pickle.load(f)
f.close()

# imprime
dt.printTree(data)

+-- ROOT
    +-- A
    |   +-- A1
    |   +-- A2
    |   +-- A3
    +-- B
    |   +-- B1
    |   +-- B2
    +-- C
        +-- C1
        +-- C2
        +-- C3 {a: 1, b: 2, c: 3}
        +-- C4 {d: 4, e: 5, f: 6}


**Ejemplo**. Recupere los subarboles desde el arbol `data`.

In [4]:
data_A = data.find('A') # la función find permite recuperar el nodo a partir de la propiedad `tag`.
data_B = data.find('B')
data_C = data.find('C')

dt.printTree(data_A)
print(' ')
dt.printTree(data_B)
print(' ')
dt.printTree(data_C)

+-- A
    +-- A1
    +-- A2
    +-- A3
 
+-- B
    +-- B1
    +-- B2
 
+-- C
    +-- C1
    +-- C2
    +-- C3 {a: 1, b: 2, c: 3}
    +-- C4 {d: 4, e: 5, f: 6}


# shlex - Herramienta para construir analizadores léxicos sencillos

[Contenido](#Contenido)

In [5]:
import shlex
x = '12.4+15.4'
s1 = shlex.split(x)
s2 = shlex.shlex(x)

In [6]:
print(s1)

['12.4+15.4']


In [7]:
print(list(s2))

['12', '.', '4', '+', '15', '.', '4']


# yylex.py - Analizador léxico para el lenguaje de prueba

[Contenido](#Contenido)

## Gramática básica

[Contenido](#Contenido)

# Ejemplos lenguaje bcc

[Contenido](#Contenido)

In [8]:
%%writefile example.txt
var n:num, i:num, j:num;

n := 5;
i := 1;

while (true) do
  {
  when (n < i) do break;
  j := 1;
  while (true) do
    {
    when (n < j) do break;
    write 'i = ': i;
    write 'j = ': j;
    j++;
    }
  i++;
  }

end

Writing example.txt


In [9]:
%%sh 
python yylex.py example.txt

+-- TOKENTABLE
    +-- VAR {lexeme: var, lineno: 0}
    +-- ID {lexeme: n, lineno: 0}
    +-- : {lexeme: :, lineno: 0}
    +-- DATATYPE {lexeme: num, lineno: 0}
    +-- , {lexeme: ,, lineno: 0}
    +-- ID {lexeme: i, lineno: 0}
    +-- : {lexeme: :, lineno: 0}
    +-- DATATYPE {lexeme: num, lineno: 0}
    +-- , {lexeme: ,, lineno: 0}
    +-- ID {lexeme: j, lineno: 0}
    +-- : {lexeme: :, lineno: 0}
    +-- DATATYPE {lexeme: num, lineno: 0}
    +-- ; {lexeme: ;, lineno: 0}
    +-- ID {lexeme: n, lineno: 2}
    +-- := {lexeme: :=, lineno: 2}
    +-- NUM {lexeme: 5, lineno: 2}
    +-- ; {lexeme: ;, lineno: 2}
    +-- ID {lexeme: i, lineno: 3}
    +-- := {lexeme: :=, lineno: 3}
    +-- NUM {lexeme: 1, lineno: 3}
    +-- ; {lexeme: ;, lineno: 3}
    +-- WHILE {lexeme: while, lineno: 5}
    +-- ( {lexeme: (, lineno: 5}
    +-- BOOL {lexeme: true, lineno: 5}
    +-- ) {lexeme: ), lineno: 5}
    +-- DO {lexeme: do, lineno: 5}
    +-- { {lexeme: {, lineno: 6}
    +-- WHEN {lexeme: when, lineno

---

In [10]:
%%writefile example.txt
var z:num;
z := 0;    
do 
  {
  z := z + 1;
  write 'do while (1 a 10) ==> ':  z;
  } 
  while (z < 10)

end

Overwriting example.txt


In [11]:
%%sh 
python yylex.py example.txt

+-- TOKENTABLE
    +-- VAR {lexeme: var, lineno: 0}
    +-- ID {lexeme: z, lineno: 0}
    +-- : {lexeme: :, lineno: 0}
    +-- DATATYPE {lexeme: num, lineno: 0}
    +-- ; {lexeme: ;, lineno: 0}
    +-- ID {lexeme: z, lineno: 1}
    +-- := {lexeme: :=, lineno: 1}
    +-- NUM {lexeme: 0, lineno: 1}
    +-- ; {lexeme: ;, lineno: 1}
    +-- DO {lexeme: do, lineno: 2}
    +-- { {lexeme: {, lineno: 3}
    +-- ID {lexeme: z, lineno: 4}
    +-- := {lexeme: :=, lineno: 4}
    +-- ID {lexeme: z, lineno: 4}
    +-- + {lexeme: +, lineno: 4}
    +-- NUM {lexeme: 1, lineno: 4}
    +-- ; {lexeme: ;, lineno: 4}
    +-- WRITE {lexeme: write, lineno: 5}
    +-- STR {lexeme: 'do while (1 a 10) ==> ', lineno: 5}
    +-- : {lexeme: :, lineno: 5}
    +-- ID {lexeme: z, lineno: 5}
    +-- ; {lexeme: ;, lineno: 5}
    +-- } {lexeme: }, lineno: 6}
    +-- WHILE {lexeme: while, lineno: 7}
    +-- ( {lexeme: (, lineno: 7}
    +-- ID {lexeme: z, lineno: 7}
    +-- < {lexeme: <, lineno: 7}
    +-- NUM {lexeme: 10,

---

In [12]:
%%writefile example.txt
write '$chs( 1 ) ==> ':  \$chs( 1 );
write '$abs( 1 ) ==> ':  \$abs( 1 );
write '$sgn( 0 ) ==> ':  \$sgn( 0 );
write '$sgn( 1 ) ==> ':  \$sgn( 1 );
end

Overwriting example.txt


In [13]:
%%sh 
python yylex.py example.txt

example.txt:Lexical error at line <0>: Unexpected symbol <\>


---

In [14]:
%%writefile example.txt
function @min:num (x:num, y:num)
   {
   when ((x < y) == true) do return x;
   return y;
   }
   
## funcion max(x, y)
function @max:num (x:num, y:num)   
   {
   if ((x < y) == false) do
      {
      return x;
      }
   else
      {                 
      return y;
      }
   }

write 'min 1 -> ': @min(1,2);
write 'max 2 -> ': @max(1,2);

end

Overwriting example.txt


In [15]:
%%sh 
python yylex.py example.txt

+-- TOKENTABLE
    +-- FUNCTION {lexeme: function, lineno: 0}
    +-- UFID {lexeme: @min, lineno: 0}
    +-- : {lexeme: :, lineno: 0}
    +-- DATATYPE {lexeme: num, lineno: 0}
    +-- ( {lexeme: (, lineno: 0}
    +-- ID {lexeme: x, lineno: 0}
    +-- : {lexeme: :, lineno: 0}
    +-- DATATYPE {lexeme: num, lineno: 0}
    +-- , {lexeme: ,, lineno: 0}
    +-- ID {lexeme: y, lineno: 0}
    +-- : {lexeme: :, lineno: 0}
    +-- DATATYPE {lexeme: num, lineno: 0}
    +-- ) {lexeme: ), lineno: 0}
    +-- { {lexeme: {, lineno: 1}
    +-- WHEN {lexeme: when, lineno: 2}
    +-- ( {lexeme: (, lineno: 2}
    +-- ( {lexeme: (, lineno: 2}
    +-- ID {lexeme: x, lineno: 2}
    +-- < {lexeme: <, lineno: 2}
    +-- ID {lexeme: y, lineno: 2}
    +-- ) {lexeme: ), lineno: 2}
    +-- == {lexeme: ==, lineno: 2}
    +-- BOOL {lexeme: true, lineno: 2}
    +-- ) {lexeme: ), lineno: 2}
    +-- DO {lexeme: do, lineno: 2}
    +-- RETURN {lexeme: return, lineno: 2}
    +-- ID {lexeme: x, lineno: 2}
    +-- ; {lexem


## Implementación de yylex.py

[Contenido](#Contenido)

In [16]:
%%sh
pygmentize -O linenos=1 -g yylex.py

0001: ###< 2016-08-28 17:04:40.182335 >###
0002: 
0003: #
0004: #  yylex.py
0005: #    analizador lexico para el lenguaje bcc
0006: #
0007: 
0008: import sys, shlex, pickle
0009: import dataTree as dt
0010: 
0011: #
0012: # analizador lexico
0013: #
0014: def yylex(filename, quiet=False):
0015: 
0016:     # estructura de datos.
0017:     #   crea un arbol para almacenar la informacion
0018:     #   generada por las distintas componentes
0019:     #   del interprete
0020:     DATA = dt.TreeNode('DATA', {'filename': filename})
0021:     SOURCECODE  = dt.SubNode(DATA, 'SOURCECODE')
0022:     TOKENTABLE  = dt.SubNode(DATA, 'TOKENTABLE')
0023:     SYNTAXTREE  = dt.SubNode(DATA, 'SYNTAXTREE')
0024:     SYMBOLTABLE = dt.SubNode(DATA, 'SYMBOLTABLE')
0025: 
0026:     # preparacion del codigo fuente.
0027:     #   sourceCode es un vector donde cada elemento
0028:     #   es una linea del codigo fuente incluyendo
0029:     #   comentarios. Despues almacena el codigo
0030:     #   en la estructura

# Gramática extendida para el lenguaje final

[Contenido](#Contenido)

---

[Contenido](#Contenido)